In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

Using TensorFlow backend.


In [11]:
# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = '/mnt/safe01/data/public/isic/train'
validation_data_dir = '/mnt/safe01/data/public/isic/test'
nb_train_samples = 4048
nb_validation_samples = 480
epochs = 50
batch_size = 16

In [3]:
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train', 'wb'),
            bottleneck_features_train)
    
    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation', 'wb'),
bottleneck_features_validation)

In [9]:
def train_top_model():
    train_data = np.load(open('bottleneck_features_train','rb'))
    train_labels = np.array(
        [0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load(open('bottleneck_features_validation', 'rb'))
    validation_labels = np.array(
        [0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


In [6]:
save_bottlebeck_features()

Found 4053 images belonging to 2 classes.
Found 482 images belonging to 2 classes.


In [12]:
train_top_model()

Train on 4048 samples, validate on 480 samples
Epoch 1/50
4048/4048 [==============================] - 2s - loss: 0.7952 - acc: 0.5170 - val_loss: 0.6893 - val_acc: 0.5333
Epoch 2/50
4048/4048 [==============================] - 2s - loss: 0.6934 - acc: 0.5363 - val_loss: 0.6861 - val_acc: 0.5563
Epoch 3/50
4048/4048 [==============================] - 2s - loss: 0.6882 - acc: 0.5338 - val_loss: 0.6879 - val_acc: 0.5625
Epoch 4/50
4048/4048 [==============================] - 2s - loss: 0.6872 - acc: 0.5546 - val_loss: 0.6789 - val_acc: 0.5833
Epoch 5/50
4048/4048 [==============================] - 2s - loss: 0.6775 - acc: 0.5702 - val_loss: 0.6818 - val_acc: 0.5625
Epoch 6/50
4048/4048 [==============================] - 2s - loss: 0.6709 - acc: 0.5973 - val_loss: 0.6916 - val_acc: 0.5625
Epoch 7/50
4048/4048 [==============================] - 2s - loss: 0.6675 - acc: 0.5879 - val_loss: 0.7066 - val_acc: 0.5312
Epoch 8/50
4048/4048 [==============================] - 2s - loss: 0.6588 - ac